In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [2]:
dataset=load_boston()

In [3]:
type(dataset)

sklearn.utils.Bunch

In [4]:
dataset

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

In [5]:
dataset.data.shape

(506, 13)

In [6]:
dataset.target.shape

(506,)

In [8]:
X_full,Y_full=dataset.data,dataset.target
n_samples=dataset.data.shape[0]
n_features=dataset.data.shape[1]

In [13]:
rng=np.random.RandomState(0)
missing_rate=0.5
n_missing_samples=int(np.floor(n_samples*n_features*missing_rate))


In [23]:
missing_features=rng.randint(0,n_features,n_missing_samples)
missing_samples=rng.randint(0,n_samples,n_missing_samples)
missing_features
missing_samples

array([213, 470, 466, ..., 501, 221, 489])

In [21]:
X_missing=X_full.copy()
Y_missing=Y_full.copy()

In [24]:
X_missing[missing_samples,missing_features]=np.nan

In [27]:
X_missing=pd.DataFrame(X_missing)

In [31]:
#使用均值和0填补缺失值
imp_mean=SimpleImputer(missing_values=np.nan,strategy="mean") 
X_missing_men=imp_mean.fit_transform(X_missing)

In [32]:
imp_0=SimpleImputer(missing_values=np.nan,strategy="constant",fill_value=0)
X_missing_0=imp_0.fit_transform(X_missing)

In [33]:
X_missing_0

array([[6.3200e-03, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 3.9690e+02,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 0.0000e+00,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 0.0000e+00,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 0.0000e+00, ..., 2.1000e+01, 0.0000e+00,
        0.0000e+00],
       [1.0959e-01, 0.0000e+00, 0.0000e+00, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [0.0000e+00, 0.0000e+00, 1.1930e+01, ..., 0.0000e+00, 3.9690e+02,
        0.0000e+00]])

In [34]:
X_missing_reg=X_missing.copy()

In [40]:
sortindex=np.argsort(X_missing_reg.isnull().sum()).values     #通过argsort接口 将列索引由小到大排列

In [51]:
for i in sortindex :
    df=X_missing_reg
    fillc=df.iloc[:,i]
    df=pd.concat([df.iloc[:,df.columns!=i],pd.DataFrame(Y_full)],axis=1)
    
    df_0=SimpleImputer(missing_values=np.nan,strategy="constant",fill_value=0).fit_transform(df)
    
    Ytrain=fillc[fillc.notnull()]
    Ytest=fillc[fillc.isnull()]
    Xtrain=df_0[Ytrain.index,:]
    Xtest=df_0[Ytest.index,:]
    
    rfc=RandomForestRegressor(n_estimators=100)
    rfc=rfc.fit(Xtrain,Ytrain)
    Y_predict=rfc.predict(Xtest)
    
    X_missing_reg.loc[X_missing_reg.iloc[:,i].isnull(),i]=Y_predict

In [52]:
X_missing_reg

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.006320,18.000,6.2910,0.00,0.445407,6.36894,65.200,4.090000,4.57,304.88,17.755,396.9000,7.2697
1,0.135833,0.000,7.0700,0.00,0.469000,6.42100,78.900,4.967100,4.46,275.36,17.800,391.4584,9.1400
2,0.027290,0.000,7.0700,0.05,0.475100,7.18500,61.100,4.967100,4.33,262.93,17.800,391.9011,4.0300
3,0.032370,76.065,2.8179,0.00,0.412380,6.99800,32.454,6.062200,3.20,255.48,18.700,394.6300,2.9400
4,0.069050,0.000,2.1800,0.00,0.458000,7.14700,47.935,6.062200,3.43,222.00,18.700,396.9000,5.3300
5,0.029850,0.000,4.0277,0.01,0.458000,6.43000,58.700,6.062200,3.00,222.00,18.700,394.6875,7.0997
6,0.182520,12.500,9.0708,0.00,0.478539,6.01200,51.176,5.560500,4.87,315.02,15.200,392.2524,12.4300
7,0.394081,6.080,7.8700,0.39,0.524000,6.18475,96.100,5.950500,6.94,311.00,17.856,396.9000,19.1500
8,0.308489,8.075,11.7530,0.00,0.524000,5.63100,83.259,6.082100,5.00,311.00,15.200,386.6300,29.9300
9,0.170040,12.500,7.8700,0.00,0.483980,6.00400,85.900,4.938802,4.52,297.68,15.200,380.6471,12.9620


In [53]:
#对所有数据进行建模，取得MSE结果
from  sklearn.model_selection import cross_val_score

X=[X_full,X_missing_men,X_missing_0,X_missing_reg]

mse=[]
std=[]

for x in X:
    estimator=RandomForestRegressor(random_state=0,n_estimators=100)
    scores=cross_val_score(estimator,x,Y_full,cv=10,scoring="neg_mean_squared_error").mean()
    mse.append(scores*-1)
    

In [54]:
mse


[21.765883307921563,
 30.088889795725503,
 38.397270450352934,
 16.179713611294105]

In [55]:
estimator.estimator_params

('criterion',
 'max_depth',
 'min_samples_split',
 'min_samples_leaf',
 'min_weight_fraction_leaf',
 'max_features',
 'max_leaf_nodes',
 'min_impurity_decrease',
 'min_impurity_split',
 'random_state')